# Рекомендация тарифов

Клиентам компании "Мегалайн" предлагают два тарифных плана: «Смарт» и «Ультра». Мы имееем данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Необходимо оптимизировать модель по метрике accuracy (минимально допустимое значение 0.75) и проверить работу модели на тестовой выборке.

## Открытие файлов и изучение общей информации

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Открываем и изучаем файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Просматриваем общую информацию

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Заменям тип данных в столбце с сообщениями на целочисленный

In [4]:
df['messages']=df['messages'].astype('int')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83,19915.42,0
1,85.0,516.75,56,22696.96,0
2,77.0,467.66,86,21060.45,0
3,106.0,745.53,81,8437.39,1
4,66.0,418.74,1,14502.75,0


Проверяем отсутствующие значения

In [5]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Отсутствующие значения не обнаружены

**Вывод:**
1. Данные изучены
2. Тип данных в столбце с сообщениями изменён на целочисленный
3. Отсутствующие значения не обнаружены
4. В данных 3214 пользователей и 5 их критериев поведения


## Разделение данных на выборки

Выделяем:
1. Целевой признак - is_ultra
2. Признаки - calls, minutes, messages, mb_used

In [6]:
target = df['is_ultra']
features = df.drop('is_ultra', axis=1)

In [7]:
print(features.shape)
print(target.shape)

(3214, 4)
(3214,)


Разделяем исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [8]:
all_features, test_features, all_target, test_target = train_test_split(features, target, test_size=0.2, random_state=123)

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(all_features, all_target, test_size = 0.25, random_state=123)

In [10]:
print(features_train.shape)
print(features_valid.shape)
print(test_features.shape)

(1928, 4)
(643, 4)
(643, 4)


## Исследование моделей

**Исследуем модель дерева решений.**

In [11]:
model = DecisionTreeClassifier(max_depth = 20, random_state=12345)
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7247278382581649


Установив значение гиперпараметра max_depth 20, мы получили значенние accuracy 0.72, что недостаточно велико. Попробуем поменять значение гиперпараметра.

In [12]:
model = DecisionTreeClassifier(max_depth = 5, random_state=12345)
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7791601866251944


Установив значение гиперпараметра max_depth 20, мы получили значенние accuracy 0.77, что удовлетворяет условию. 

**Исследуем модель случайного леса.**

In [13]:
model = RandomForestClassifier(n_estimators=50, random_state=12345)
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7853810264385692


Установив значение гиперпараметра n_estimators 50, мы получили значенние accuracy 0.78, что удовлетворяет условию.Попробуем поменять значение гиперпараметра для поиска ещё бОльшего значения.

In [14]:
model = RandomForestClassifier(n_estimators=20, random_state=12345)
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.776049766718507


Установив значение гиперпараметра n_estimators 20, мы получили значенние accuracy 0.77, что удовлетворяет условию.Попробуем поменять значение гиперпараметра для поиска ещё бОльшего значения.

In [15]:
model = RandomForestClassifier(n_estimators=300, random_state=12345)
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7962674961119751


Установив значение гиперпараметра n_estimators 300, мы получили значенние accuracy 0.79, что удовлетворяет условию и больше предыдущего. 

Реализуем подбор параметра с помощью цикла

In [16]:
best_model = None
best_result = 0

for est in range(1, 30):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_estimator = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result)     
print("Параметр наилучшей модели на валидационной выборке:", best_estimator)

Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Параметр наилучшей модели на валидационной выборке: 6


**Исследуем модель логистической регрессии.**

In [17]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7340590979782271


Получаем значение меньше 0.75. Данную модель не рассматриваем.

**Вывод:**
1. Исследованы разные модели.
2. Рассмотрены разные варианты гиперпараметров.
3. По результатам исследования лучшее значение accuracy показала модель случайного леса (0.79)

## Проверка модели на тестовой выборке

In [19]:
model_test = RandomForestClassifier(n_estimators=6,random_state=12345)
                                      
model_test.fit(features_train, target_train)
predictions = model_test.predict(test_features)
accuracy = accuracy_score(test_target, predictions)
print('Точность модели на тестовой выборке:', accuracy)

Точность модели на тестовой выборке: 0.7729393468118196


**Вывод:**
При проверке избранной модели на тестовой выборке, она также показала accuracy  больше 0.75, что удовлетворяет условию.

# Общий вывод

1. Данные изучены
2. Данные по необходимости преобразованы в нужные типы
3. Данные разделены на выборки
4. Качества различных моделей исследованы со сменой гиперпараметров. Лучшей моделью оказался случайный лес, показав accuracy ближе к 80
5. Качество модели проверено на тестовой выборке